# Computing for Economists: Data Wrangling





In this module, we will have you work through the following datasets to analyze social connectedness in Europe. This module is inspired by [Bailey et al. (2020)](https://pages.stern.nyu.edu/~jstroebe/PDF/BJRKSS_EuroSCI.pdf).

In [ ]:
### Import Modules
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

## Tidying the Data

1. The `data` folder contains data from the following sources. Read each of the data files into Python as Pandas DataFrames.
  - https://ec.europa.eu/eurostat/en/web/products-datasets/-/EDAT_LFSE_04 
    - This data contains the percent of population by educational attainment level, sex, and NUTS 2 regions in the European Union.
  - https://ec.europa.eu/eurostat/web/products-datasets/-/nama_10r_2hhinc
    - The data contains the income of households by NUTS 2 region.  
  - https://ec.europa.eu/eurostat/web/products-datasets/product?code=demo_r_pjanaggr3
    - Population by age, group, sex, and NUTS 2 region.
  - https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts
    - NUTS region shapefiles (use the geopandas library to read it as a GeoDataFrame).
  - https://media.githubusercontent.com/media/social-connectedness-index/euro_sci/master/_intermediate_data/geo_distance_dat.csv
    - Distance between pairs of NUTS 2 regions constructed by [Bailey et al. (2020)](https://pages.stern.nyu.edu/~jstroebe/PDF/BJRKSS_EuroSCI.pdf).
  - `gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv` from https://data.humdata.org/dataset/social-connectedness-index
    - This file contains the social connectedness index between NUTS 2 regions and all Global Administrative Areas (GADM). We will only consider NUTS 2 to NUTS 2 connections. Social connectedness is a snapshot of Facebook links as of July 2019.
    - Note: this file is very large so we recommend reading the file directly into the notebook at the following link: https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/5de92e01-606e-4e4d-ad7c-4a3d493a0cc3/download/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv


In [ ]:
edu_data = pd.read_csv("data/estat_edat_lfse_04.tsv", sep='\t')

inc_data = pd.read_csv("data/estat_nama_10r_2hhinc.tsv", sep='\t')

pop_data = pd.read_csv("data/estat_demo_r_pjanaggr3.tsv", sep='\t')

shapefile = gpd.read_file("data/NUTS_RG_20M_2021_3035.shp")

dist_data = pd.read_csv("data/geo_distance_dat.csv")

sci_url = "https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/5de92e01-606e-4e4d-ad7c-4a3d493a0cc3/download/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv"
sci = pd.read_csv(sci_url, sep='\t')


2. Clean the education data (`estat_edat_lfse_04.tsv`) and put it into "tidy" format. Check that there is a unique non-missing key for each row. The resulting data should have five columns and 299,088 rows.

Below we do the same for the income data (`estat_nama_10r_2hhinc.tsv`) and the population data (`estat_demo_r_pjanaggr3.tsv`).

In [ ]:
inc_data[['freq', 'unit', 'direct', 'na_item', 'geo']] = inc_data['freq,unit,direct,na_item,geo\TIME_PERIOD'].str.split(',', expand=True)

inc_data = (
    inc_data
    .drop(columns=["freq", "freq,unit,direct,na_item,geo\TIME_PERIOD"])
    .melt(id_vars=['unit', 'direct', 'na_item', 'geo'], value_name="value", var_name="year")
    .assign(
        value = lambda x: pd.to_numeric(x["value"], errors="coerce"),
        year = lambda x: x["year"].astype(int)
    )
)

pop_data[['freq', 'unit', 'sex', 'age', 'geo']] = pop_data['freq,unit,sex,age,geo\TIME_PERIOD'].str.split(',', expand=True)

pop_data = (
    pop_data
    .drop(columns=["freq", "freq,unit,sex,age,geo\TIME_PERIOD"])
    .melt(id_vars=['unit', 'sex', 'age', 'geo'], value_name="pop", var_name="year")
    .assign(
        pop = lambda x: pd.to_numeric(x["pop"], errors="coerce"),
        year = lambda x: x["year"].astype(int)
    )
)

3. Inner join the percent of the population age 25 to 64 in a NUTS 2 region receiving a tertiary education with the income per capita in purchasing power standard (PPS). Then inner join the education and income data with population data for the total population (for sex and age). Only include data at the NUTS 2 level.

  To obtain the income per capita in PPS, the national account should be the "balance of primary incomes/national income, net" and the direction of flow should be the "balance".

In [ ]:
edu_data = (
    edu_data
    .query("age == 'Y25-64'")
    .query("isced11 == 'ED5-8'")
    .drop(columns=["age", "isced11"])
    .rename(columns={"pct": "pct_tertiary_edu"})
)

inc_data = (
    inc_data
    .query("unit == 'PPS_EU27_2020_HAB'")
    .query("na_item == 'B5N'")
    .query("direct == 'BAL'")
    .drop(columns=["unit", "na_item", "direct"])
    .rename(columns={"value": "inc_per_capita"})
)

pop_data = (
    pop_data
    .query("age == 'TOTAL'")
    .query("sex == 'T'")
    .drop(columns=["unit", "sex", "age"])
    .rename(columns={"pop": "population"})
)

# join datasets

# remove non-NUTS 2 level rows

4. For each pair of NUTS 2 regions, obtain the absolute difference in the two variables constructed in problem 3 in 2019. Drop the French islands (starts with NUTS 2 region code "FRY").


5. Now we will merge the the 2019 data with the SCI data and the pairwise distance data. Create a dataframe with one row for each pair of NUTS 2 regions (as two separate columns) and the following variables:

*   Log scaled SCI
*   Log distance (add 1 to avoid zeros)
*   Percent tertiary education for User and Friend NUTS 2
*   Income per capita for User and Friend NUTS 2
*   Population for User and Friend NUTS 2
*   Absolute difference in tertiary education between User and Friend NUTS 2
*   Absolute difference in income per capita between User and Friend NUTS 2

  Note that the SCI data contains GADM areas, please only keep the NUTS 2 to NUTS 2 rows. Keeps all rows in the SCI data.


## Descriptive Statistics


1.   Each country is represented by the first two letters of the NUTS 2 index. Present a country-level population-weighted summary table for the percent people with tertiary education and income per capita in 2019. Include the mean, standard deviation, min, and max. Round all numeric outputs to two decimal places. We have provided you with the functions to compute weighted means and standard deviations.

In [1]:
# Define custom aggregation functions for weighted mean and weighted std
def weighted_mean(x, weights):
    mask = ~np.isnan(x) & ~np.isnan(weights)  # Filter out NaNs
    x, weights = x[mask], weights[mask]
    denom = np.sum(weights)
    if np.sum(weights) == 0:
        return np.NaN
    else:
        return np.sum(x * weights) / np.sum(weights)

def weighted_std(x, weights):
    mask = ~np.isnan(x) & ~np.isnan(weights)  # Filter out NaNs
    x, weights = x[mask], weights[mask]
    n = len(x)
    if n == 0:
      return np.NaN
    else:
      mean = weighted_mean(x, weights)
      numerator = np.sum(weights * (x - mean)**2)
      denominator = (n - 1) / n * np.sum(weights)
      return np.sqrt(numerator / denominator)
    
# your code 

2. The SCI is scaled from 1 to 1,000,000 and measures the relative probability of a Facebook friendship link ([source](https://data.humdata.org/dataset/social-connectedness-index?)). If a number is two times higher, then a friendship link between two users from those NUTS 2 regions is twice as likely. Calculate  the mean, std, min, and max of log(scaled SCI) by country and merge it with the summary table from the previous part.

3. Compute the (unweighted) correlation between log(scaled SCI), log(distance), user region education and income, as well as differences in education and income between NUTS 2 regions. Is it higher or lower than what you expected?

## Data Visualization

1. We will ask you to plot two maps of social connections of (1) a country (of your choice) and (2) a NUTS 2 region (of your choice) to all other NUTS 2 regions. Remember the French islands should be dropped (these islands are far away from continental Europe and will therefore make the maps harder to visualize). We have provided the majority of the code to perform the mapping and we will explain how the aggregation of SCI to the country level is performed.

  * For the chosen country, aggregate the SCI to produce country to NUTS 2 region measures. Scaled SCI can be aggregated correctly using population shares following [SCI documentation](https://data.humdata.org/dataset/social-connectedness-index). For countries $i, j$ with regions $\{r_i\}, \{r_j\}$, aggregate SCI is given by:

  $$SCI_{i,j} = \sum_{\{r_i\}}\sum_{\{r_j\}} PopShare_{r_i} \times PopShare_{r_j} \times SCI_{r_i, r_j}$$

  where $PopShare_{r_i}$ is the share of country $i$'s population in region $r_i$.

  * Merge the aggregated SCI data with the GeoDataFrame (keep only NUTS 2 regions) for the applicable geographies.
  * Construct the bins for the map legend by first selecting a baseline SCI cutoff (e.g. 20th percentile of the **total data** that you are working with). Then calculate the subsequent cutoffs as $2\times$, $3\times$, $5\times$, $10\times$, $25\times$, $100\times, 1000\times$ that of the baseline cutoff. Feel free to use another legend system as long as there is justification.
  * Map directly using the GeoPandas library, or feel free to explore interactive maps in Plotly.

You only need to fill in the correct DataFrames, then call on the mapping function for your two plots. Feel free to make adjustments as needed.

In [ ]:
# Create population shares for country data
ctry_pop_share = (
    # DataFrame with 2019 data
    .assign(country = lambda df: df.index.str[:2])
    .assign(country_pop = lambda df: df.groupby('country')['population'].transform('sum'))
    .assign(country_pop_share = lambda df: df['population'] / df['country_pop'])
)

# Merge population shares to calculate aggregated SCI
nuts2country = (
    pd.merge(
        # DataFrame with pairs of NUTS 2 Regions[['user_loc', 'fr_loc', 'scaled_sci']],
        ctry_pop_share,
        left_on = 'user_loc',
        right_index = True,
        how = 'inner'
    )
    .assign(sci_agg = lambda df: df['country_pop_share'] * df['scaled_sci'])
    .groupby(['country', 'fr_loc'])['sci_agg'].sum()
    .reset_index()
)

In [ ]:
def map_sci_nuts2(df, geo_id, region, map_var, baseline=0.2):
    """
    Plot map of SCI between specified region and all NUTS 2 regions.

    Parameters
    ----------
    df : DataFrame
      Dataframe aggregated to the correct level for SCI
    geo_id : string
      Name of variable which is the geography who's SCI is being plotted
    region : string
      Name of region which is being plotted (e.g. 'FR' or 'BÊ35')
    map_var : string
      Name of the variable being mapped (e.g. sci)
    baseline : float between 0 and 1
      Default for smallest bin is 20th percentile of data

    Returns
    -------
    Nothing. Plots the map directly.
    """
    # Calculate bins based on total data
    map_bins = df[map_var].quantile(baseline) * np.array([0, 1, 2, 3, 5, 10, 25, 100, 1000, 10e9])

    map_bin_labels = ['< 1x (Overall 20th percentile)', '1-2x', '2-3x', '3-5x',
                      '5-10x', '10-25x', '25-100x', '100x-1000x', '>= 1000x']

    df['sci_bins'] = pd.cut(df[map_var], bins=map_bins, labels=map_bin_labels, include_lowest=True)

    # Merge SCI data to shape files for NUTS 2
    merge_shape = (
        shapefile[shapefile['LEVL_CODE'] == 2]
        .merge(
            df[df[geo_id] == region],
            left_on = "NUTS_ID",
            right_on = "fr_loc",
            how = 'inner'
        )
    )

    # Plot Map
    legend_kwds = {
        'loc': 'upper left', # Position of legend
        'bbox_to_anchor': (0, 1)
    }

    fig, ax = plt.subplots(figsize=(10, 10))
    merge_shape.plot(column='sci_bins', ax=ax, cmap='PuBu', legend=True, legend_kwds = legend_kwds, linewidth=1, edgecolor='0.6')
    ax.set_title("Scaled SCI Between " + region + " and All NUTS 2 Regions")
    ax.set_axis_off()
    plt.show()

In [2]:
# Plot country map

# Plot NUTS 2 to NUTS 2 region map

2. Show the relationship between log SCI and log distance in a figure. Highlight the points of at least one user geography (country or NUTS 2). Fit a curve to the data and plot it along with the scatterplot.

3. Visualize the distributions of log SCI by income bin (e.g. low, medium, high) of the user region. Comment.

***Bonus:*** Explore the data and feel free to visualize whatever you would like. Make sure you understand the variables and compute summary statistics.

# Submission ⭐

Congratulations you finished the module! We will handle all exercise submissions on GitHub Classroom. Please push your changes to your team repository folder on GitHub Classroom.